## PDI - Phuture DeFi Index (Multi-chain) - Methodology Template

## Configuring packages

In [5]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [6]:
min_mcap =  0.5e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = "decentralized-finance-defi"

# Calculating max slippage based on liquidity threshold
liq = 5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                min_mcap,
                                min_weight,
                                max_weight,
                                255,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                coingecko_category)

## Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [7]:

results,liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    remove_category_assets=['the-graph','staked-ether','injective-protocol','chainlink','seedify-fund']
    )
liquidity

Excluding chainflip, pricing data available only for 92 < 180 days
Excluding stakewise-v3-oseth, pricing data available only for 74 < 180 days
0xbec243c995409e6520d7c41e404da5deba4b209b avalanche
0xa0b862f60edef4452f25b4160f177db44deb6cf1 arbitrum-one
0x7205705771547cf79201111b4bd8aaf29467b9ec polygon-pos
0xaf8ca653fa2772d58f4368b0a71980e9e3ceb888 optimistic-ethereum
0xd58d345fd9c82262e087d2d0607624b410d88242 arbitrum-one
0xe3322702bedaaed36cddab233360b939775ae5f1 polygon-pos
0xd7c49cee7e9188cca6ad8ff264c1da2e69d4cf3b ethereum
0x9046d36440290ffde54fe0dd84db8b1cfee9107b optimistic-ethereum
0x9eaf8c1e34f05a589eda6bafdf391cf6ad3cb239 base
0x77f56cf9365955486b12c4816992388ee8606f0e polygon-pos
0xf0cb2dc0db5e6c66b9a70ac27b06b878da017028 arbitrum-one
0xd3cc9d8f3689b83c91b7b59cab4946b063eb894a ethereum
0x1e1085efaa63edfe74aad7c05a28eae4ef917c3f fantom
0x753d224bcf9aafacd81558c32341416df61d3dac arbitrum-one
Dropping dydx because a price feed is unavailable
Dropping chaingpt because a price fee

,spot price,delivery price,slippage,blockchain
id,,,,
stargate-finance,1.495902,1.501766e+00,0.003920,ethereum
lido-dao,0.327884,3.283749e-01,0.001497,ethereum
convex-finance,0.206931,2.069165e-01,-0.000071,ethereum
tbtc,0.000019,1.934603e-05,-0.000979,arbitrum-one
uniswap,0.134775,1.346324e-01,-0.001057,arbitrum-one
...,...,...,...,...
unifi-protocol-dao,0.143360,1.306643e-02,-0.908856,binance-smart-chain
stp-network,16.208078,8.516550e-01,-0.947455,ethereum
vvs-finance,265721.733218,1.031608e+04,-0.961177,ethereum


## Show Results

In [8]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain_with_highest_liq
id,,,,,,
uniswap,UNI,5558733034,0.287130,74,0xfa7f8980b0f1e64a2062791cc3b0871572f1f7f0,arbitrum-one
lido-dao,LDO,2693182772,0.139114,36,0x5a98fcbea516cf06857215779fd812ca3bef1b32,ethereum
maker,MKR,1872015957,0.096697,25,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,ethereum
aave,AAVE,1353038235,0.069890,18,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,ethereum
havven,SNX,1160139566,0.059926,16,0x8700daec35af8ff88c16bdf0418774cb3d7599b4,optimistic-ethereum
gnosis,GNO,814042826,0.042049,11,0x6810e776880c02933d47db1b9fc05908e5386b96,ethereum
pancakeswap-token,CAKE,662060332,0.034198,9,0x0e09fabb73bd3ade0a17ecc321fd13a19e81ce82,binance-smart-chain
frax-share,FXS,649178066,0.033533,9,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,ethereum
curve-dao-token,CRV,623987471,0.032231,9,0x172370d5cd63279efa6d502dab29171933a610af,polygon-pos
